In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
167,AFG,Asia,Afghanistan,2020-06-25,29715.0,234.0,639.0,21.0,763.326,6.011,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
276,ALB,Europe,Albania,2020-06-25,2114.0,67.0,47.0,2.0,734.589,23.282,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
449,DZA,Africa,Algeria,2020-06-25,12248.0,172.0,869.0,8.0,279.309,3.922,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-24,France,161704.0,0.0,29752.0,0.0,0.0,0.0
0,2020-06-24,China,84726.0,0.0,4651.0,0.0,0.0,0.0
0,2020-06-24,Italy,239192.0,0.0,34723.0,0.0,0.0,0.0
0,2020-06-24,Spain,247055.0,0.0,28345.0,0.0,0.0,0.0
0,2020-06-24,United States,2379103.0,0.0,121836.0,0.0,0.0,0.0
0,2020-06-24,World,9404599.0,0.0,481509.0,0.0,0.0,0.0
0,2020-06-24,United Kingdom,307571.0,0.0,43010.0,0.0,0.0,0.0
0,2020-06-24,Germany,192009.0,0.0,8934.0,0.0,0.0,0.0
0,2020-06-24,Iran,212920.0,0.0,9977.0,0.0,0.0,0.0
0,2020-06-24,Turkey,191302.0,0.0,5023.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-25,France,161704.0,161348.0,29752.0,29731.0,356.0,21.0
0,2020-06-25,China,84726.0,84673.0,4651.0,4640.0,53.0,11.0
0,2020-06-25,Italy,239192.0,239410.0,34723.0,34644.0,-218.0,79.0
0,2020-06-24,Spain,247055.0,247086.0,28345.0,28327.0,-31.0,18.0
0,2020-06-25,United States,2379103.0,2381361.0,121836.0,121979.0,-2258.0,-143.0
0,2020-06-25,World,9404599.0,9400295.0,481509.0,482468.0,4304.0,-959.0
0,2020-06-25,United Kingdom,307571.0,306862.0,43010.0,43081.0,709.0,-71.0
0,2020-06-25,Germany,192009.0,192079.0,8934.0,8927.0,-70.0,7.0
0,2020-06-25,Iran,212920.0,212501.0,9977.0,9996.0,419.0,-19.0
0,2020-06-25,Turkey,191302.0,191657.0,5023.0,5025.0,-355.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")